<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/HR_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Required Libraries

In [1]:
!pip install -qU langchain transformers sentence-transformers llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 38.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.7/228.7 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00


In [3]:
%pip install --upgrade --quiet  docx2txt

  Preparing metadata (setup.py) ... done


In [35]:
!pip install chromadb

In [8]:
%pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.9/283.9 kB 4.5 MB/s eta 0:00:00


In [26]:
%pip install rank_bm25

## Load Data

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## Import Required Dependencies

In [12]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.retrievers import ParentDocumentRetriever
from langchain.chains import RetrievalQA
from langchain.llms import LlamaCpp


In [2]:
import os
import tqdm
path  = "/content/drive/MyDrive/hrdocs"
file_list = os.listdir(path)
documents = []
for files in tqdm.tqdm(file_list):
  if files.endswith(".pdf"):
    pdf_path = os.path.join(path,files)
    pdf_loader = PyPDFLoader(pdf_path)
    documents.extend(pdf_loader.load())
  elif files.endswith(".docx"):
    docx_path = os.path.join(path,files)
    docx_loader = Docx2txtLoader(docx_path)
    documents.extend(docx_loader.load())
  else:
    continue



100%|██████████| 104/104 [00:16<00:00,  6.15it/s]


In [3]:
len(documents)

173

In [4]:
print(documents[10].metadata)

{'source': '/content/drive/MyDrive/hrdocs/Company-travel-policy (1).docx'}


## Prepare the document to have processed metadata to be helpful in metadata filtering

In [5]:
for doc in documents:
  doc.metadata['title'] = doc.metadata['source'].split(".")[0].split("/")[-1].split(" ")[0]

In [6]:
print(documents[10].metadata)

{'source': '/content/drive/MyDrive/hrdocs/Company-travel-policy (1).docx', 'title': 'Company-travel-policy'}


In [7]:
print(documents[100].metadata)

{'source': '/content/drive/MyDrive/hrdocs/Security-Policy-2-Account-Management.docx', 'title': 'Security-Policy-2-Account-Management'}


## Instantiate the embedding model

In [15]:
model_name = "BAAI/bge-small-en-v1.5"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings":True}
embeddings = HuggingFaceBgeEmbeddings(model_name=model_name,
                                      model_kwargs=model_kwargs,
                                      encode_kwargs=encode_kwargs,
                                      )

## Instantiate the LLM Model

#### We can also downlaod the model file by executing the below command

* pip install huggingface-hub

* ! huggingface-cli download TheBloke/zephyr-7B-beta-GGUF zephyr-7b-beta.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False


In [9]:
model_path ="/content/drive/MyDrive/Mistral_beta/zephyr-7b-beta.Q4_K_M.gguf"
llms = LlamaCpp(streaming=True,
                model_path=model_path,
                max_tokens=500,
                temperature=0.75,
                top_p=1,
                model_kwargs={"gpu_layers":0,"stream":True},
                verbose=True,
                n_threads=int(os.cpu_count()/2),
                n_ctx=4096)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


## Instantiate the Vectorstore

In [16]:
vectorstore = Chroma(embedding_function=embeddings,
                     persist_directory="/content/drive/MyDrive/Vectorstore/chroma_full",
                     collection_name="full_documents")

## Instantiate the Text Splitter

In [17]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

## Instantiate the ParentDocumentRetriver

In [18]:
store = InMemoryStore()
#
retriever = ParentDocumentRetriever(vectorstore=vectorstore,
                                    docstore=store,
                                    parent_splitter=parent_splitter,
                                    child_splitter=child_splitter,)

## Load Documents

In [19]:
retriever.add_documents(documents)
vectorstore.persist()

## Check if the Documents have been uploaded successfully

In [36]:
vectorstore.get().keys()

dict_keys(['ids', 'embeddings', 'metadatas', 'documents', 'uris', 'data'])

In [37]:
len(vectorstore.get()['documents'])

3008

## Instantiate the Keyword Search Model

In [27]:
from langchain.retrievers import BM25Retriever,EnsembleRetriever
from langchain.retrievers import ContextualCompressionRetriever
#
parent_documents = parent_splitter.split_documents(documents)
bm25_retriever = BM25Retriever.from_documents(parent_documents)
bm25_retriever.k=10

## Instantiate ReRanker - Cross Encoders

In [31]:
from __future__ import annotations
from typing import Dict, Optional, Sequence
from langchain.schema import Document
from langchain.pydantic_v1 import Extra, root_validator

from langchain.callbacks.manager import Callbacks
from langchain.retrievers.document_compressors.base import BaseDocumentCompressor

from sentence_transformers import CrossEncoder
# from config import bge_reranker_large

class BgeRerank(BaseDocumentCompressor):
    model_name:str = 'BAAI/bge-reranker-large'
    """Model name to use for reranking."""
    top_n: int = 3
    """Number of documents to return."""
    model:CrossEncoder = CrossEncoder(model_name)
    """CrossEncoder instance to use for reranking."""

    def bge_rerank(self,query,docs):
        model_inputs =  [[query, doc] for doc in docs]
        scores = self.model.predict(model_inputs)
        results = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
        return results[:self.top_n]


    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    def compress_documents(
        self,
        documents: Sequence[Document],
        query: str,
        callbacks: Optional[Callbacks] = None,
    ) -> Sequence[Document]:
        """
        Compress documents using BAAI/bge-reranker models.

        Args:
            documents: A sequence of documents to compress.
            query: The query to use for compressing the documents.
            callbacks: Callbacks to run during the compression process.

        Returns:
            A sequence of compressed documents.
        """
        if len(documents) == 0:  # to avoid empty api call
            return []
        doc_list = list(documents)
        _docs = [d.page_content for d in doc_list]
        results = self.bge_rerank(query, _docs)
        final_results = []
        for r in results:
            doc = doc_list[r[0]]
            doc.metadata["relevance_score"] = r[1]
            final_results.append(doc)
        return final_results

## Create a Contextual Compression Pipeline

In [41]:
from langchain.document_transformers import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers import ContextualCompressionRetriever
from langchain.document_transformers import LongContextReorder
#
source_name = "corporate-social-responsibility-policy"
vs_retriever = vectorstore.as_retriever(search_kwargs={"k":10,"filter":{'title':source_name},})
#
retriever.search_kwargs = {"k":10,"filter":{'title':source_name},}
#
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever,vs_retriever,retriever],
                                       weight=[0.5,0.5,0.1])
#
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
#
reordering = LongContextReorder()
#
reranker = BgeRerank()
#
pipeline_compressor = DocumentCompressorPipeline(transformers=[redundant_filter,reordering,reranker])
#
compression_pipeline = ContextualCompressionRetriever(base_compressor=pipeline_compressor,
                                                      base_retriever=ensemble_retriever,
                                                      metadata={"title":source_name},
)

## Define the Retrieval Q&A Engine

In [42]:
from langchain.chains import RetrievalQA
#
qa = RetrievalQA.from_chain_type(llm=llms,
                                 chain_type="stuff",
                                 retriever=compression_pipeline,
                                 return_source_documents=True)


## Pretty Print Documnets

In [63]:
def pretty_print_docs(docs):
  print(
      f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n + {d.page_content}" for i,d in enumerate(docs)])
  )

## Invoke the query engine

In [64]:
pretty_print_docs(retriever.get_relevant_documents(query))

Document 1:

 + INTRODUCTION   
 
Social responsibility allows the company's business interests to be reconciled  
with the legitimate interests of the different stakeholders that may be affected,  
and also assumes the impact of the company's activities on the community in  
general  and the environment.  
 
The present Corporate Social Responsibility Policy (hereinafter, “CSR Policy”)  
intends to establish a reference framework, from which any ACME CORP  
company can develop and strengthen socially responsible behavior, regardless  
of the way — convent ional and/or digital — the business is carried out and the  
country  it operates  in, locally  adapting  any actions  that are necessary  for 
compliance with  it. 
 
 
  GOALS   
 
ACME CORP  defines  its social  responsibility  as a "voluntary  and strategic  
commitment that entails attempting to achieve business targets while complying  
strictly with its legal and contractual obligations, applying criteria of equity and  
stabi

#### Test the compression_pipeline

In [67]:
pretty_print_docs(compression_pipeline.get_relevant_documents(query))

Document 1:

 + INTRODUCTION   
 
Social responsibility allows the company's business interests to be reconciled  
with the legitimate interests of the different stakeholders that may be affected,  
and also assumes the impact of the company's activities on the community in  
general  and the environment.  
 
The present Corporate Social Responsibility Policy (hereinafter, “CSR Policy”)  
intends to establish a reference framework, from which any ACME CORP  
company can develop and strengthen socially responsible behavior, regardless  
of the way — convent ional and/or digital — the business is carried out and the  
country  it operates  in, locally  adapting  any actions  that are necessary  for 
compliance with  it. 
 
 
  GOALS   
 
ACME CORP  defines  its social  responsibility  as a "voluntary  and strategic  
commitment that entails attempting to achieve business targets while complying  
strictly with its legal and contractual obligations, applying criteria of equity and  
stabi

## Ask Query

In [44]:
query ="How ACME CORP has defined its social responsibility?"
response = qa(query)

In [45]:
response

{'query': 'How ACME CORP has defined its social responsibility?',
 'result': ' ACME CORP defines its social responsibility as a "voluntary and strategic commitment that entails attempting to achieve business targets while complying strictly with its legal and contractual obligations, applying criteria of equity and stability to relationships with its stakeholders, thus contributing to meeting the current and future needs of society."',
 'source_documents': [_DocumentWithState(page_content='INTRODUCTION   \n \nSocial responsibility allows the company\'s business interests to be reconciled  \nwith the legitimate interests of the different stakeholders that may be affected,  \nand also assumes the impact of the company\'s activities on the community in  \ngeneral  and the environment.  \n \nThe present Corporate Social Responsibility Policy (hereinafter, “CSR Policy”)  \nintends to establish a reference framework, from which any ACME CORP  \ncompany can develop and strengthen socially res

In [46]:
response.keys()

dict_keys(['query', 'result', 'source_documents'])

In [52]:
response['source_documents'][0].page_content

'INTRODUCTION   \n \nSocial responsibility allows the company\'s business interests to be reconciled  \nwith the legitimate interests of the different stakeholders that may be affected,  \nand also assumes the impact of the company\'s activities on the community in  \ngeneral  and the environment.  \n \nThe present Corporate Social Responsibility Policy (hereinafter, “CSR Policy”)  \nintends to establish a reference framework, from which any ACME CORP  \ncompany can develop and strengthen socially responsible behavior, regardless  \nof the way — convent ional and/or digital — the business is carried out and the  \ncountry  it operates  in, locally  adapting  any actions  that are necessary  for \ncompliance with  it. \n \n \n  GOALS   \n \nACME CORP  defines  its social  responsibility  as a "voluntary  and strategic  \ncommitment that entails attempting to achieve business targets while complying  \nstrictly with its legal and contractual obligations, applying criteria of equity and  

In [53]:
response['source_documents'][0].metadata

{'source': '/content/drive/MyDrive/hrdocs/corporate-social-responsibility-policy.pdf',
 'page': 2,
 'title': 'corporate-social-responsibility-policy',
 'relevance_score': 0.9973786}

In [54]:
Final_Response = {"query":response['query'],
                  "response":response['result'],
                  "Source":[doc.page_content for doc in response['source_documents']],
                  "Source_metadata":[doc.metadata['source'] for doc in response['source_documents']],
                  "Source_relevance_score":[doc.metadata['relevance_score'] for doc in response['source_documents']],
                  }

#### Final Response

In [55]:
Final_Response

{'query': 'How ACME CORP has defined its social responsibility?',
 'response': ' ACME CORP defines its social responsibility as a "voluntary and strategic commitment that entails attempting to achieve business targets while complying strictly with its legal and contractual obligations, applying criteria of equity and stability to relationships with its stakeholders, thus contributing to meeting the current and future needs of society."',
 'Source': ['INTRODUCTION   \n \nSocial responsibility allows the company\'s business interests to be reconciled  \nwith the legitimate interests of the different stakeholders that may be affected,  \nand also assumes the impact of the company\'s activities on the community in  \ngeneral  and the environment.  \n \nThe present Corporate Social Responsibility Policy (hereinafter, “CSR Policy”)  \nintends to establish a reference framework, from which any ACME CORP  \ncompany can develop and strengthen socially responsible behavior, regardless  \nof the 

## Load the Vectorstore

In [68]:
vs =  Chroma(embedding_function=embeddings,
                     persist_directory="/content/drive/MyDrive/Vectorstore/chroma_full",
                     collection_name="full_documents")

In [69]:
len(vs.get()['documents'])

3008

## Reload the retriever

In [70]:
retriever_load = ParentDocumentRetriever(vectorstore=vs,
                                    docstore=store,
                                    parent_splitter=parent_splitter,
                                    child_splitter=child_splitter,)

## Check if the reload has happened properly

In [71]:
pretty_print_docs(retriever_load.get_relevant_documents(query))

Document 1:

 + Corporate social responsibility Acme Corp policy

This Corporate Social Responsibility Acme Corp policy template is ready to be tailored to your Acme Corp’s needs and should be considered a starting point for setting up your social responsibility employment policies.

 Policy brief & purpose

Our Corporate Social Responsibility (CSR) Acme Corp policy refers to our responsibility toward our environment. Our Acme Corp’s existence is not lonely. It's part of a bigger system of people, values, other organizations and nature. The social responsibility of a business is  to give back to the world just as it gives to us.

What is corporate social responsibility?

Our Corporate Social Responsibility (CSR) Acme Corp policy outlines our efforts to give back to the world as it gives to us.

Scope

This policy applies to our Acme Corp and its subsidiaries. It may also refer to suppliers and partners.

Policy elements

We want to be a responsible business that meets the highest stand

#### Reload the contextual Compression Pipeline

In [72]:
#
source_name = "transport-policy"
vs_retriever = vs.as_retriever(search_kwargs={"k":10,"filter":{'title':source_name},})
#
retriever_load.search_kwargs = {"k":10,"filter":{'title':source_name},}
#
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever,vs_retriever,retriever_load],
                                       weight=[0.5,0.5,0.1])
#
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
#
reordering = LongContextReorder()
#
reranker = BgeRerank()
#
pipeline_compressor = DocumentCompressorPipeline(transformers=[redundant_filter,reordering,reranker])
#
compression_pipeline = ContextualCompressionRetriever(base_compressor=pipeline_compressor,
                                                      base_retriever=ensemble_retriever,
                                                      metadata={"title":source_name},
)

#### Check if we are able to load documents from compressor pipeline

In [73]:
query ="What is the transportaion policy or procedure?"
pretty_print_docs(compression_pipeline.get_relevant_documents(query))

Document 1:

 + 4.	Policy /Procedure guidelines	4

		5.	Review and Revision:	6

		6.	Non-compliance and consequences	6




























Objective



Acme Corp endeavors to constantly provide the best of facilities and benefits to its employees. The Transport policy provides guidelines to employees and the Transport management team with regards to the eligibility and procedures to be followed.
----------------------------------------------------------------------------------------------------
Document 2:

 + Acme Corp


HR Policy 









Transport policy



Policy Effective Date: <DD/MM/YY>





No part of this documentation may be reproduced or transmitted in any form or by any means, electronic or mechanical, including photocopying or recording, for any purpose without express written permission of the CEO of Acme Corp.



© 2021, Acme Corp. All Rights Reserved 



Revision History

 

Ver No.

Change Description

Prepared By

Reviewed By

Approved By

Date













 